In [3]:
from helper_functions import unzip_data

In [5]:
unzip_data("nlp-getting-started.zip")

In [6]:
import pandas as pd

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [13]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [17]:
print(len(train_df))
print(len(test_df))

7613
3263


In [75]:
import numpy as np
def get_random_sample():
    n = np.random.randint(len(train_df)-1)
    row = train_df.iloc[n:n+1,:].itertuples()
    for x in row:
        print('Target :',x.target)
        print('Text :',x.text)

In [88]:
get_random_sample()

Target : 1
Text : Typhoon Soudelor approaches after 7 killed 2 missing in floods in PhilippinesÛ_ http://t.co/8rDXcfgQEm #Typhoon


In [100]:
X = train_df['text'].to_numpy()
y = train_df['target'].to_numpy()

In [101]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_targets, val_targets = train_test_split(X,y,test_size=0.1,random_state=42)

In [102]:
len(train_sentences),len(val_sentences)


(6851, 762)

In [103]:
train_sentences[:5]

array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
       'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
       "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
       "I'm On Fire.  http://t.co/WATsmxYTVa",
       "More than 40 families affected by the fatal outbreak of Legionnaires' disease in Edinburgh are to sue two comp... http://t.co/vsoXioOy78"],
      dtype=object)

In [104]:
train_targets[:5]

array([0, 1, 0, 0, 1])

In [107]:
max_vocab_len = 10000
max_len = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [108]:
from tensorflow.keras.layers import TextVectorization
text_vectorizer = TextVectorization(max_tokens = max_vocab_len,
                                    output_mode = 'int',
                                    output_sequence_length = max_len)

In [109]:
text_vectorizer.adapt(train_sentences)

2024-01-23 22:42:04.440563: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [110]:
text_vectorizer(['Typhoon Soudelor approaches'])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 476,  759, 2183,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [111]:
top_5 = text_vectorizer.get_vocabulary()[:5]
bottom_5 = text_vectorizer.get_vocabulary()[-5:]
print("top 5:",top_5)
print("Bottom 5:",bottom_5)

top 5: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5: ['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']
